In [26]:
import pandas.io.data as web
from datetime import datetime
import random
import math
import urllib
from lxml import html
import csv

In [27]:
url = "https://blockchain.info/charts/market-price/?showDataPoints=false&timespan=all&show_header=true&daysAverageString=1&scale=0&format=csv&address="
bitcoin_prices = urllib.urlopen(url).read()

outfilename = 'btc_prices.csv'
outfile = open(outfilename,'w')
outfile.write(bitcoin_prices)
outfile.close()

In [28]:
#testing 
with open(outfilename, 'rb') as f:
    reader = csv.reader(f)
    for row in reader:
        print row[0]
        print date_object.date()
        break


03/01/2009 18:15:05
2016-05-21


In [30]:
#reformatting org file
dates = []
prices = []
with open(outfilename, 'rb') as f:
    reader = csv.reader(f)
    for row in reader:
        if float(row[1]) > 0:
            date_object = datetime.strptime(row[0], '%d/%m/%Y %H:%M:%S')
            dates.append(date_object.strftime("%Y-%m-%d"))
            prices.append(float(row[1]))
dates
prices

[0.0769,
 0.074,
 0.0688,
 0.0667,
 0.066899,
 0.0664,
 0.066,
 0.066889,
 0.0665,
 0.066499,
 0.065,
 0.065,
 0.0648,
 0.069,
 0.06497,
 0.0649,
 0.0629,
 0.0634,
 0.0613,
 0.0629,
 0.064,
 0.06185,
 0.06201,
 0.0624,
 0.06201,
 0.062,
 0.064999,
 0.06201,
 0.0641,
 0.175,
 0.0619,
 0.0609,
 0.0609,
 0.062599,
 0.0634,
 0.0633,
 0.0628,
 0.063,
 0.06281,
 0.0624,
 0.062279,
 0.062206,
 0.06271,
 0.06219,
 0.06192,
 0.061999,
 0.061999,
 0.0614,
 0.062,
 0.06301,
 0.0633,
 0.0638,
 0.088,
 0.12001,
 0.12,
 0.1301,
 0.099,
 0.095,
 0.105,
 0.119,
 0.109,
 0.1045,
 0.103,
 0.1024,
 0.103,
 0.109,
 0.107,
 0.10901,
 0.19,
 0.15,
 0.19,
 0.19,
 0.19001,
 0.191,
 0.1919,
 0.199,
 0.1975,
 0.1955,
 0.275,
 0.23601,
 0.2399,
 0.29,
 0.499999,
 0.37,
 0.3368,
 0.2667,
 0.24,
 0.251,
 0.3,
 0.299,
 0.2828,
 0.2827,
 0.28,
 0.289999,
 0.283,
 0.289,
 0.29,
 0.282,
 0.2879,
 0.28299,
 0.28299,
 0.289,
 0.284399,
 0.28461,
 0.279,
 0.275,
 0.225,
 0.25,
 0.2553,
 0.2589,
 0.235,
 0.225,
 0.226,
 0

In [46]:
import random

def selectSampleIndex(listSize, sampleSize, maxSeqLen):
    if listSize <= 0 or sampleSize <= 0 or listSize < sampleSize:
        return []
    
    resSample = set()
    while len(resSample) < sampleSize:
        pick = random.randint(0, listSize-maxSeqLen)
        resSample.add(pick)
    res = list(resSample)
    res.sort()
    return res

def getDiffValue(lastDayValue, targetDayValue):
    return float(targetDayValue - lastDayValue)

def getDiff(lastDayValue, targetDayValue):
    return getDiffValue(lastDayValue, targetDayValue)/float(targetDayValue)


In [47]:
maxSeqLen = 20
marginDays = 5
minGain = 0.05
numSamples = 200

In [48]:
resIdx = selectSampleIndex(len(prices), numSamples, maxSeqLen)
resIdx[0:10], resIdx[-10:] #see first and last 10 index

([1, 2, 6, 7, 28, 29, 40, 44, 53, 59],
 [2014, 2016, 2032, 2034, 2036, 2041, 2057, 2064, 2075, 2076])

In [50]:
outfilename = 'btc_train_data.csv'
outfile = open(outfilename,'w')

#print header
header = 'date,' + 'day'+',day'.join([`num+1` for num in xrange(maxSeqLen-marginDays)])+',dayTarget'+',diffVal'+',diff'+',isBullish'+'\n'
outfile.write(header)

for idx in resIdx:
    isBullish = 0    
    diff = getDiff(prices[idx+(maxSeqLen-marginDays)-1], prices[idx+maxSeqLen])
    if diff > minGain:
        isBullish = 1
    elif diff < (-1*minGain):
        isBullish = -1
    else:
        isBullish = 0
    #print idx, tsla_prices[idx:idx+10], tsla_prices[idx+15], good, diff
    #print tsla_prices[idx:idx+maxSeqLen-5], tsla_prices[idx+maxSeqLen], isBullish
    result = prices[idx:idx+maxSeqLen-marginDays]  #prices of sequence of days
    result.insert(0, dates[idx+(maxSeqLen-marginDays)-1]) #date
    result.append(prices[idx+maxSeqLen]) #price of the day target
    result.append(getDiffValue(prices[idx+(maxSeqLen-marginDays)-1], prices[idx+maxSeqLen])) #diff value
    result.append(diff) #diff 
    result.append(isBullish) #isbullish
    trainset = ",".join([`x` for x in result])
    outfile.write(trainset)
    outfile.write('\n')

outfile.close()


In [25]:
'day'+',day'.join([`num+1` for num in xrange(maxSeqLen)])

'day1,day2,day3,day4,day5,day6,day7,day8,day9,day10,day11,day12,day13,day14,day15,day16,day17,day18,day19,day20'

In [30]:
%alias_magic --line whereami pwd


Created `%whereami` as an alias for `%pwd`.


In [32]:
%pwd

u'D:\\Work\\data\\stockprices'